In [1]:
import h5py
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import json
import os
import pickle
import random
import re
import pysam
import tensorflow as tf
from basenji import seqnn
from basenji import stream

2023-07-24 03:38:17.891841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2023-07-24 03:38:17.891887: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from akita_utils.seq_gens import split_span, symmertic_insertion_seqs_gen
from akita_utils.dna_utils import dna_1hot, hot1_rc
from akita_utils.utils import ut_dense

In [4]:
from akita_utils.stats_utils import calculate_scores

In [5]:
from akita_utils.h5_utils import initialize_stat_output_h5, write_stat_metrics_to_h5

In [6]:
genome_fasta="/project/fudenber_735/genomes/mm10/mm10.fa" 

In [7]:
models_dir="/project/fudenber_735/tensorflow_models/akita/v2/models"

In [8]:
seq_coords_df = pd.read_csv("./test.tsv", sep="\t")

In [9]:
seq_coords_df

,m0_SCD,m0_SSD,boundary_index,boundary_strength_200000,chrom,boundary_end,index,log2_insulation_score_200000,num_ctcf,score_2,...,m7_SCD,m7_SSD,m7_INS-16,m7_INS-64,start,end,orientation,flank_bp,spacer_bp,background_index
0,1.06400,-0.174400,0,1.014,chr1,4410000,0,-0.4258,2,803.0,...,0.63040,-0.049530,0.000549,-0.000977,4403267,4403286,>,30,0,0
1,0.05704,0.000491,0,1.014,chr1,4410000,1,-0.4258,2,834.0,...,0.09717,-0.021440,0.000153,0.000244,4405891,4405910,>,30,0,0
2,34.53000,9.086000,1,1.003,chr1,4780000,4,-0.4893,10,974.0,...,21.36000,5.980000,-0.010990,-0.017090,4770055,4770074,>,30,0,0
3,0.23600,-0.045400,1,1.003,chr1,4780000,5,-0.4893,10,833.0,...,0.27150,-0.014984,0.001465,0.001465,4770180,4770199,>,30,0,0
4,0.18000,-0.095700,1,1.003,chr1,4780000,6,-0.4893,10,816.0,...,0.08540,0.026260,-0.000488,0.000000,4770867,4770886,>,30,0,0
5,0.49410,-0.138200,1,1.003,chr1,4780000,7,-0.4893,10,815.0,...,1.20800,-0.007202,0.002930,0.001465,4773435,4773454,>,30,0,0
6,0.47460,-0.201500,1,1.003,chr1,4780000,8,-0.4893,10,853.0,...,0.30980,-0.005642,-0.001465,-0.000488,4773687,4773706,>,30,0,0
7,0.32400,-0.046780,1,1.003,chr1,4780000,9,-0.4893,10,802.0,...,0.58940,0.029900,0.000977,0.000000,4775739,4775758,>,30,0,0
8,2.45900,-0.972700,1,1.003,chr1,4780000,10,-0.4893,10,821.0,...,1.12800,0.041300,0.000732,0.000488,4777268,4777287,>,30,0,0
9,0.39820,-0.093800,1,1.003,chr1,4780000,11,-0.4893,10,802.0,...,0.28880,0.033230,-0.000977,-0.000977,4777427,4777446,>,30,0,0


In [10]:
seq_coords_df.columns

Index(['m0_SCD', 'm0_SSD', 'boundary_index', 'boundary_strength_200000',
       'chrom', 'boundary_end', 'index', 'log2_insulation_score_200000',
       'num_ctcf', 'score_2', 'span', 'boundary_start', 'strand', 'm0_INS-16',
       'm0_INS-64', 'm1_SCD', 'm1_SSD', 'm1_INS-16', 'm1_INS-64', 'm2_SCD',
       'm2_SSD', 'm2_INS-16', 'm2_INS-64', 'm3_SCD', 'm3_SSD', 'm3_INS-16',
       'm3_INS-64', 'm4_SCD', 'm4_SSD', 'm4_INS-16', 'm4_INS-64', 'm5_SCD',
       'm5_SSD', 'm5_INS-16', 'm5_INS-64', 'm6_SCD', 'm6_SSD', 'm6_INS-16',
       'm6_INS-64', 'm7_SCD', 'm7_SSD', 'm7_INS-16', 'm7_INS-64', 'start',
       'end', 'orientation', 'flank_bp', 'spacer_bp', 'background_index'],
      dtype='object')

In [11]:
model_index = 0
head_index = 1

model_dir = models_dir + "/f" + str(model_index) + "c0/train/"
model_file = model_dir + "model" + str(head_index) + "_best.h5"
params_file = model_dir + "params.json"

In [12]:
genome_open = pysam.Fastafile(genome_fasta)

In [13]:
# read model parameters
with open(params_file) as params_open:
    params = json.load(params_open)
params_train = params["train"]
params_model = params["model"]

batch_size = 4

# load model
seqnn_model = seqnn.SeqNN(params_model)
seqnn_model.restore(model_file, head_i=head_index)

(rc, shifts) = (False, [0])

seqnn_model.build_ensemble(rc, shifts)
seq_length = int(params_model["seq_length"])

2023-07-24 03:38:29.578716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2023-07-24 03:38:29.578753: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-24 03:38:29.578797: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b02-16.hpc.usc.edu): /proc/driver/nvidia/version does not exist
2023-07-24 03:38:29.578994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1310720, 4)  0           []                               
                                ]                                                                 
                                                                                                  
 stochastic_reverse_complement   ((None, 1310720, 4)  0          ['sequence[0][0]']               
 (StochasticReverseComplement)  , ())                                                             
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1310720, 4)  0           ['stochastic_reverse_complement[0
 ift)                                                            ][0]']                     

In [14]:
# seq generator

In [15]:
out_dir = "./TEST"

In [16]:
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
plot_map = False

In [17]:
max(seq_coords_df.background_index.unique())

0

In [18]:
scd_stats_initialize = ["SCD", "SSD", "INS-16", "INS-64"]

In [28]:
stats_out = initialize_stat_output_h5(
    out_dir,
    model_file,
    scd_stats_initialize,
    seq_coords_df)

OSError: Unable to create file (unable to truncate a file which is already open)

In [20]:
for key in stats_out:
    print(key)

SCD_h1_m0_t0
SCD_h1_m0_t1
SCD_h1_m0_t2
SCD_h1_m0_t3
SCD_h1_m0_t4
SCD_h1_m0_t5
SSD_h1_m0_t0
SSD_h1_m0_t1
SSD_h1_m0_t2
SSD_h1_m0_t3
SSD_h1_m0_t4
SSD_h1_m0_t5
alt_INS-16_h1_m0_t0
alt_INS-16_h1_m0_t1
alt_INS-16_h1_m0_t2
alt_INS-16_h1_m0_t3
alt_INS-16_h1_m0_t4
alt_INS-16_h1_m0_t5
alt_INS-64_h1_m0_t0
alt_INS-64_h1_m0_t1
alt_INS-64_h1_m0_t2
alt_INS-64_h1_m0_t3
alt_INS-64_h1_m0_t4
alt_INS-64_h1_m0_t5
background_index
boundary_end
boundary_index
boundary_start
boundary_strength_200000
chrom
end
flank_bp
index
log2_insulation_score_200000
m0_INS-16
m0_INS-64
m0_SCD
m0_SSD
m1_INS-16
m1_INS-64
m1_SCD
m1_SSD
m2_INS-16
m2_INS-64
m2_SCD
m2_SSD
m3_INS-16
m3_INS-64
m3_SCD
m3_SSD
m4_INS-16
m4_INS-64
m4_SCD
m4_SSD
m5_INS-16
m5_INS-64
m5_SCD
m5_SSD
m6_INS-16
m6_INS-64
m6_SCD
m6_SSD
m7_INS-16
m7_INS-64
m7_SCD
m7_SSD
num_ctcf
orientation
ref_INS-16_h1_m0_t0
ref_INS-16_h1_m0_t1
ref_INS-16_h1_m0_t2
ref_INS-16_h1_m0_t3
ref_INS-16_h1_m0_t4
ref_INS-16_h1_m0_t5
ref_INS-64_h1_m0_t0
ref_INS-64_h1_m0_t1
ref_INS-64_h

In [21]:
background_file = f"/project/fudenber_735/tensorflow_models/akita/v2/analysis/mouse_backgrounds/m{model_index}_background_seqs.fa"

In [22]:
background_seqs = []
with open(background_file, "r") as f:
    for line in f.readlines():
        if ">" in line:
            continue
        background_seqs.append(dna_1hot(line.strip()))

In [23]:
backgrounds_predictions = seqnn_model.predict(np.array(background_seqs), batch_size=6)

2/2 [==============================] - 9s 3s/step


In [24]:
backgrounds_predictions.shape

(10, 130305, 6)

In [25]:
preds_stream = stream.PredStreamGen(
    seqnn_model,
    symmertic_insertion_seqs_gen(seq_coords_df, background_seqs, genome_open),
    batch_size,
)

In [26]:
for si in range(len(seq_coords_df)):
    
    print(f"sequence {si}")
    
    bg_index = seq_coords_df.iloc[si].background_index
    
    prediction_matrix = preds_stream[si]
    reference_prediction_matrix = backgrounds_predictions[bg_index, :, :]
    
    write_stat_metrics_to_h5(prediction_matrix,
                                reference_prediction_matrix,
                                stats_out,
                                si,
                                head_index,
                                model_index,
                             2,
                            scd_stats_initialize)    
    

sequence 0
3/3 [==============================] - 9s 2s/step
sequence 1
sequence 2
sequence 3
sequence 4
sequence 5
sequence 6
sequence 7
sequence 8
sequence 9


In [27]:
for key in stats_out:
    print(key, stats_out[key][0])

SCD_h1_m0_t0 1.524
SCD_h1_m0_t1 1.941
SCD_h1_m0_t2 1.16
SCD_h1_m0_t3 1.078
SCD_h1_m0_t4 1.306
SCD_h1_m0_t5 1.021
SSD_h1_m0_t0 -0.526
SSD_h1_m0_t1 -0.6016
SSD_h1_m0_t2 0.03937
SSD_h1_m0_t3 -0.01752
SSD_h1_m0_t4 -0.1543
SSD_h1_m0_t5 -0.1422
alt_INS-16_h1_m0_t0 -0.05322
alt_INS-16_h1_m0_t1 -0.04138
alt_INS-16_h1_m0_t2 -0.06934
alt_INS-16_h1_m0_t3 -0.0774
alt_INS-16_h1_m0_t4 -0.0636
alt_INS-16_h1_m0_t5 -0.07477
alt_INS-64_h1_m0_t0 -0.0578
alt_INS-64_h1_m0_t1 -0.05386
alt_INS-64_h1_m0_t2 -0.08844
alt_INS-64_h1_m0_t3 -0.09314
alt_INS-64_h1_m0_t4 -0.0782
alt_INS-64_h1_m0_t5 -0.084
background_index 0
boundary_end 4410000
boundary_index 0
boundary_start 4400000
boundary_strength_200000 1.014
chrom b'chr1'
end 4403286
flank_bp 30
index 0
log2_insulation_score_200000 -0.4258
m0_INS-16 0.001831
m0_INS-64 0.001221
m0_SCD 1.064
m0_SSD -0.1744
m1_INS-16 0.0002441
m1_INS-64 0.0004883
m1_SCD 0.3281
m1_SSD -0.06573
m2_INS-16 0.0001221
m2_INS-64 0.0004883
m2_SCD 1.383
m2_SSD -0.2712
m3_INS-16 0.006714
m3